<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>To Restart AWS EC2 Instance by given tag using unSkript actions.</strong></div>
</center><center>
<h2 id="Restart-EC2-Instance">Restart EC2 Instance By Given Tag</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<ol>
<li>Filter AWS EC2 Instances by tag</li>
<li>Restart AWS EC2 Instance</li>
<li>Get AWS Instance Details</li>
</ol>

<h3 id="Filter-AWS-EC2-Instances-by-tag">Filter AWS EC2 Instances by tag</h3>
<p>In this action, we search for all the instances from AWS for a given tag and region and return a list of instances.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>tag_key,&nbsp;tag_value,&nbsp;region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>instance_list</code></p>
</blockquote>

In [3]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_filter_ec2_by_tags_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_ec2_by_tags(handle, tag_key: str, tag_value: str, region: str) -> List:
    """aws_filter_ec2_by_tags Returns an array of instances matching tags.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Array of instances matching tags.
    """
    # Input param validation.

    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations",
                            Filters=[{'Name': 'tag:' + tag_key, 'Values': [tag_value]}])

    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            result.append(instance['InstanceId'])
    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "region",
    "tag_key": "tag_key",
    "tag_value": "tag_value"
    }''')
task.configure(outputName="instance_list")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ec2_by_tags, lego_printer=aws_filter_ec2_by_tags_printer, hdl=hdl, args=args)

<h3 id="Restart-AWS-EC2-Instances">Restart AWS EC2 Instances</h3>
<p>Here we will use unSkript&nbsp;<strong>Restart AWS EC2 Instances </strong>action. This action is used to restart the instances which we get using the above step 1. We pass the instances IDs list to step 2.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>instance_ids, region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>restart_instance</code></p>
</blockquote>

In [4]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_restart_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_restart_ec2_instances(handle, instance_ids: List, region: str) -> Dict:
    """aws_restart_instances Restarts instances.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.reboot_instances(InstanceIds=instance_ids)
    return res


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "instance_ids": "instance_list",
    "region": "region"
    }''')
task.configure(outputName="restart_instance")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_restart_ec2_instances, lego_printer=aws_restart_ec2_instances_printer, hdl=hdl, args=args)

<h3 id="Get-AWS-Instance-Details">Get AWS Instance Details</h3>
<p>Here we will use unSkript <strong>Get AWS Instance Details</strong> action to get the details of the instances. This action is used to get details of instances that we received in step 1.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>instance_id, region</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>instance_details</code></p>
</blockquote>

In [5]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint
from beartype import beartype

def aws_get_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_get_instance_details(
    handle,
    instance_id: str,
    region: str,
) -> Dict:

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instances(
        Filters=[{"Name": "instance-id", "Values": [instance_id]}])
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            instances.append(instance)

    return instances[0]



task = Task(Workflow())
task.configure(printOutput=True)
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_id": "iter_item",
    "region": "region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "instance_list",
    "iter_parameter": "instance_id"
    }''')
task.configure(outputName="instance_details")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_instance_details, lego_printer=aws_get_instances_printer, hdl=hdl, args=args)

<h3>Modify Step-3</h3>
<p>In this action, we sort the output from step-3 and present the details of the instance in the good table.</p>

In [7]:
import pprint
from pydantic import BaseModel, Field
from typing import Dict
from tabulate import tabulate
task.configure(printOutput=True)

output = instance_details
instance_list = instance_list
def Instance_Details(output,instance_list: list):
    data1 = []
    Header = ""
    for instance_id in instance_list:
        if instance_id in output.keys():
            output1 = output[instance_id]
            if isinstance(output1, (list, tuple)):
                for item in output1:
                    print(f'item: {item}')
            elif isinstance(output1, dict):
                for key,value in output1.items():
                    if isinstance(value, (list)):
                        pass
                    else:
                        if key == "InstanceId":
                            Header = value
                        data1.append([key,value])
                print(f'\n\033[1m Table for Instance ID : {Header} \033[0;0m')
                print(tabulate(data1))
            else:
                print(f'Output for {task.name}')
                print(output1)
Instance_Details(output,instance_list)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS legos to restart the AWS EC2 instances and get the details. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>